### Basic Chatbot with Langgraph (GRAPH API)

In [52]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [53]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [54]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [55]:
from langchain_groq import ChatGroq

import httpx


# llm = init_chat_model("groq:llama3-8b-8192")
# Create SSL-safe Groq client
http_client = httpx.Client(verify=False)
llm = ChatGroq(
    model="llama3-8b-8192",
    api_key=os.getenv("GROQ_API_KEY"),
    http_client=http_client
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1324d9a90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1324dae90>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'), http_client=<httpx.Client object at 0x1324d96d0>)

In [56]:
# node functionality
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

In [57]:
graph_builder = StateGraph(State)
graph_builder.add_node("llmchatbot", chatbot)
graph_builder.add_edge(START, "llmchatbot")
graph_builder.add_edge("llmchatbot", END)

graph = graph_builder.compile()

In [58]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Error displaying graph: {e}")
    pass

Error displaying graph: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [59]:
response = graph.invoke({"messages": ["Hi"]})
response["messages"][-1].content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [60]:
for event in graph.stream({"messages": ["Can you analyze?", "What is the weather like today?"]}):
    for values in event.values():
        print(values["messages"][-1].content)

I'm just a large language model, I don't have the ability to directly access real-time weather information. However, I can try to help you find out the weather forecast for your location or a specific location you're interested in.

To do this, I can use a combination of natural language processing and web scraping techniques to retrieve the current weather conditions and forecast from a reliable online weather source, such as the National Weather Service (NWS) or a popular weather app.

If you can provide me with the name of the city, state, or zip code you're interested in, I can try to provide you with the current weather conditions and forecast.

Alternatively, you can also use a weather app or website to get the latest weather information.


In [61]:
from langchain_tavily import TavilySearch
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


def search_tavily(query: str) -> str:
    """Search for information using Tavily API."""
    import requests
    import os

    return requests.post("https://api.tavily.com/search",
                         json={"api_key": os.getenv("TAVILY_API_KEY"),
                               "query": query, "max_results": 3},
                         verify=False).json()


tool = search_tavily("What is langchain?")
tool

{'query': 'What is langchain?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://python.langchain.com/docs/introduction/',
   'title': 'Introduction | 🦜️   LangChain',
   'content': "*   Build a simple LLM application with chat models and prompt templates *   How to migrate from legacy LangChain agents to LangGraph *   How to use chat models to call tools *   How to pass tool outputs to chat models *   **`langchain`**: Chains, agents, and retrieval strategies that make up an application's cognitive architecture. However, these guides will help you quickly accomplish common tasks using chat models, vector stores, and other common LangChain components. LangChain is part of a rich ecosystem of tools that integrate with our framework and build on top of it. If you're looking to get up and running quickly with chat models, vector stores, or other LangChain components from a specific provider, check out our growing list of integrations.",
   'score'

In [62]:
def multiply(a: int, b: int) -> int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b

In [63]:
tools = [search_tavily, multiply]

In [64]:
llm_with_tools = llm.bind_tools(tools)
llm_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1324d9a90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1324dae90>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'), http_client=<httpx.Client object at 0x1324d96d0>), kwargs={'tools': [{'type': 'function', 'function': {'name': 'search_tavily', 'description': 'Search for information using Tavily API.', 'parameters': {'properties': {'query': {'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiplies two integers.', 'parameters': {'properties': {'a': {'description': 'The first integer.', 'type': 'integer'}, 'b': {'description': 'The second integer.', 'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [65]:
# StateGraph with tools
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition


def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))


builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", END)

graph = builder.compile()

In [66]:
response = graph.invoke({"messages": ["What is the recent AI news?"]})
response["messages"][-1].content

'{"query": "recent AI news", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.artificialintelligence-news.com/", "title": "AI News | Latest AI News, Analysis & Events", "content": "SoftBank chief: Forget AGI, ASI will be here within 10 years · Anthropic deploys AI agents to audit models for safety · Sam Altman: AI will cause job losses and", "score": 0.7458619, "raw_content": null}, {"url": "https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=ASWzDAgZFafvyL3Q1nMgJuxlttRR_Hf5LedJGhSQdhjfAe1HeHKOoRbwUkJF&gaa_ts=6883cd43&gaa_sig=EyWiaY-P_uUMDWpolFN_ZbGYk7HRfmfSM0vwqHp1Qrkw1-ZSfWTh0DKsLVBTsIM48tMxH6pib0muPyi2tJ4Aww%3D%3D", "title": "Artificial Intelligence - Latest AI News and Analysis - WSJ.com", "content": "More in Artificial Intelligence · Google Revenue Soars on AI Boom, and Investors Eye Spending Surge · Walmart Taps Instacart Executive to Lead Its AI Ambitions.", "score": 0.71113056, "raw_content": null}, {"url": "https://techcrunch.com/category/a

In [67]:
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What is the recent AI news?
================================== Ai Message ==================================
Tool Calls:
  search_tavily (eq2kj9kdc)
 Call ID: eq2kj9kdc
  Args:
    query: recent AI news
================================= Tool Message =================================
Name: search_tavily

{"query": "recent AI news", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.artificialintelligence-news.com/", "title": "AI News | Latest AI News, Analysis & Events", "content": "SoftBank chief: Forget AGI, ASI will be here within 10 years · Anthropic deploys AI agents to audit models for safety · Sam Altman: AI will cause job losses and", "score": 0.7458619, "raw_content": null}, {"url": "https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=ASWzDAgZFafvyL3Q1nMgJuxlttRR_Hf5LedJGhSQdhjfAe1HeHKOoRbwUkJF&gaa_ts=6883cd43&gaa_sig=EyWiaY-P_uUMDWpolFN_ZbGYk7HRfmfSM0vwqHp1Qrkw1

In [68]:
response = graph.invoke({"messages": [
                        "What is the recent AI news and What is 2 x 3 "]})

In [69]:
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What is the recent AI news and What is 2 x 3 
================================== Ai Message ==================================
Tool Calls:
  search_tavily (19y3sanvz)
 Call ID: 19y3sanvz
  Args:
    query: recent AI news
  multiply (7rnwfdc1s)
 Call ID: 7rnwfdc1s
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: search_tavily

{"query": "recent AI news", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.artificialintelligence-news.com/", "title": "AI News | Latest AI News, Analysis & Events", "content": "SoftBank chief: Forget AGI, ASI will be here within 10 years · Anthropic deploys AI agents to audit models for safety · Sam Altman: AI will cause job losses and", "score": 0.7463059, "raw_content": null}, {"url": "https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=ASWzDAhfkJYqUWJ88syrMCaOd1UHXiQ6oufaeteud_u

### React Agent Architecture


In [74]:

# StateGraph with tools
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition


def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))


builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", "tool_calling_llm")


graph = builder.compile()

In [75]:

response = graph.invoke({"messages": [
    "What is the recent AI news and then What is 2 x 3 ?"]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What is the recent AI news and then What is 2 x 3 ?
================================== Ai Message ==================================
Tool Calls:
  search_tavily (ksd946whe)
 Call ID: ksd946whe
  Args:
    query: recent AI news
  multiply (ftyg8yfzq)
 Call ID: ftyg8yfzq
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: search_tavily

{"query": "recent AI news", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.artificialintelligence-news.com/", "title": "AI News | Latest AI News, Analysis & Events", "content": "SoftBank chief: Forget AGI, ASI will be here within 10 years · Anthropic deploys AI agents to audit models for safety · Sam Altman: AI will cause job losses and", "score": 0.7458619, "raw_content": null}, {"url": "https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=ASWzDAjz7IVJt4Fi9SNhWyLPtTBTQoEz1t6dH

In [83]:
response = graph.invoke({"messages": ["Hi My name is AAyush"]})
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi My name is AAyush
================================== Ai Message ==================================
Tool Calls:
  search_tavily (cswaj8z6f)
 Call ID: cswaj8z6f
  Args:
    query: AAyush
================================= Tool Message =================================
Name: search_tavily

{"query": "AAyush", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://en.wikipedia.org/wiki/Aayush_Sharma", "title": "Aayush Sharma - Wikipedia", "content": "Aayush Sharma (born 26 October 1990) is an Indian actor who works in Hindi films. He made his film debut with brother-in-law Salman Khan's production Loveyatri", "score": 0.7795405, "raw_content": null}, {"url": "https://www.amritocean.com/wellness-spa/aayush-hydrothermal/", "title": "Aayush Hydrothermal Therapy Florida - Amrit Ocean Resort", "content": "Explore transformative wellness at Amrit Ocean Resort's Aayush Hydrothermal

In [84]:
response = graph.invoke({"messages": ["What is my name?"]})
print(response["messages"][-1].content)

I apologize for the confusion, but it seems that the tool call id "kqwamfre5" did not yield a relevant result. Based on the conversation, I understand that you are asking for your name, and the tool call id "7ddx8bbp4" yielded a list of YouTube videos and a wikiHow page.

Since the tool call id "kqwamfre5" did not yield a relevant result, I will respond directly without using a tool.

Unfortunately, I don't have the ability to know your name, as it was not provided. If you'd like to provide your name, I'd be happy to know it and address you by name in our conversation.


In [85]:
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What is my name?
================================== Ai Message ==================================
Tool Calls:
  search_tavily (7ddx8bbp4)
 Call ID: 7ddx8bbp4
  Args:
    query: What is my name?
================================= Tool Message =================================
Name: search_tavily

{"query": "What is my name?", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.youtube.com/watch?v=y4tydaltDFY", "title": "KindlyNxsh - WHAT IS MY NAME? (Official Music Video) - YouTube", "content": "Share your videos with friends, family, and the world.", "score": 0.47606868, "raw_content": null}, {"url": "https://www.youtube.com/watch?v=_dyvXcv3ej0", "title": "What's My Name (from Descendants 2) (Official Video) - YouTube", "content": "What's My Name (from Descendants 2) (Official Video)\n\nDisneyMusicVEVO\n2449889 likes\n325711551 views\n14 Jun 2017\n\"Descendants/Zombi

### Adding Memory in Agentic Graph

In [86]:
# StateGraph with tools
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))


builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", "tool_calling_llm")


graph = builder.compile(checkpointer=memory)

In [89]:
config = {"configurable": {"thread_id": "1"}}
response = graph.invoke({"messages": ["Hi My name is ABINAYA"]}, config=config)
response["messages"][-1].content

'You are Abhinaya, a 2-time award-winning Indian actress and model, known for your work in Tamil, Telugu, Kannada, and Malayalam films.'

In [90]:
response = graph.invoke(
    {"messages": ["Hi what is my name?"]}, config=config)
response["messages"][-1].content

'Your name is ABINAYA.'

In [ ]:
response = graph.invoke(
    {"messages": ["Hi do u remember my name?"]}, config=config)
response["messages"][-1].content

'Yes, I remember! Your name is Abinaya!'

### Streaming in Langraph

In [91]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [98]:
def superbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

In [100]:
graph = StateGraph(State)

graph.add_node("superbot", superbot)

graph.add_edge(START, "superbot")
graph.add_edge("superbot", END)

llm
graph_builder = graph.compile(checkpointer=memory)

In [101]:
config = {"configurable": {"thread_id": "3"}}

for chunk in graph_builder.stream(
        {"messages": ["Hi My name is ABINAYA and I like Home decor"]}, config=config, stream_mode="updates"):
    print(chunk)

{'superbot': {'messages': [AIMessage(content="Nice to meet you, Abinaya! Home decor is such a fascinating topic! What is it about home decor that you enjoy the most? Is it the process of designing and arranging spaces, or do you have a particular style or era that you're drawn to?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 46, 'total_tokens': 101, 'completion_time': 0.075518362, 'prompt_time': 0.00654337, 'queue_time': 0.001475652000000001, 'total_time': 0.082061732}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1e7126c5-0eb8-48e5-ad74-3bf5e817e5cd-0', usage_metadata={'input_tokens': 46, 'output_tokens': 55, 'total_tokens': 101})]}}


In [102]:
for chunk in graph_builder.stream(
        {"messages": ["Hi My name is ABINAYA and I like Home decor"]}, config=config, stream_mode="values"):
    print(chunk)

{'messages': [HumanMessage(content='Hi My name is ABINAYA and I like Home decor', additional_kwargs={}, response_metadata={}, id='42ed97a5-da07-42fc-97e3-f8903d2673e9'), HumanMessage(content='Hi My name is ABINAYA and I like Home decor', additional_kwargs={}, response_metadata={}, id='c47a074b-65b9-4797-993c-2403bb1c46c2'), AIMessage(content="Nice to meet you, Abinaya! Home decor is such a fascinating topic! What is it about home decor that you enjoy the most? Is it the process of designing and arranging spaces, or do you have a particular style or era that you're drawn to?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 46, 'total_tokens': 101, 'completion_time': 0.075518362, 'prompt_time': 0.00654337, 'queue_time': 0.001475652000000001, 'total_time': 0.082061732}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1e7126c5-0eb8-48e

In [104]:
config = {"configurable": {"thread_id": "4"}}

for chunk in graph_builder.stream(
        {"messages": ["Hi My name is ABINAYA and I like Home decor"]}, config=config, stream_mode="updates"):
    print(chunk)

for chunk in graph_builder.stream(
        {"messages": ["I like a traditional and transitional styles "]}, config=config, stream_mode="values"):
    print(chunk)

{'superbot': {'messages': [AIMessage(content="Hi Abinaya!\n\nIt's great to meet you! Home decor is a fascinating topic. What is it about home decor that you enjoy the most? Is it the creative process of designing a space, the art of mixing and matching different styles, or the joy of making a space feel cozy and inviting?\n\nDo you have a favorite style or era of home decor that inspires you, such as modern, minimalist, vintage, or bohemian?\n\nLet's chat about your love for home decor and see if we can discover some common interests or find some inspiration for your own home decor projects!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 377, 'total_tokens': 497, 'completion_time': 0.117826008, 'prompt_time': 0.042676811, 'queue_time': 0.0017943090000000009, 'total_time': 0.160502819}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='